In [52]:
import pandas as pd
import re

#creating a pandas dataframe out of the csv file
traindf = pd.read_csv('../Data/08.Pubmed_PDFs_011620.csv')
testdf = pd.read_csv('../Data/04.Pubmed_PDFs_011320.csv')

out = [x[:13] for x in testdf["Identifiers"]]
#print(out)

subdf = traindf[~traindf["Identifiers"].isin(out)]
print(subdf.shape)
subdf

(88, 14)


,DateTime,Keyword,Title,URL,Abstract,Description,Details,ShortDetails,Resource,Type,Identifiers,Db,EntrezUID,Properties
0,011620-185708,"(""machine learning"" OR ""deep learning"" OR ""art...",Evaluating risk prediction models for adults w...,/pubmed/31940350,BACKGROUND: The ability to predict risk allows...,"Di Tanna GL, Wirtz H, Burrows KL, Globe G.",PloS one. doi: 10.1371/journal.pone.0224135. J...,PloS one. 2020,PubMed,Journal Article,PMID:31940350,pubmed,31940350,create date: 2020/01/15 | first author: Di Tan...
1,011620-185710,"(""machine learning"" OR ""deep learning"" OR ""art...",Automated Cardiovascular Pathology Assessment ...,/pubmed/31939003,Cardiac magnetic resonance imaging provides hi...,"Lindsey T, Lee JJ.",Journal of digital imaging. doi: 10.1007/s1027...,Journal of digital imaging. 2020,PubMed,Journal Article,PMID:31939003,pubmed,31939003,create date: 2020/01/14 | first author: Lindsey T
2,011620-185718,"(""machine learning"" OR ""deep learning"" OR ""art...",Physiological Assessment of Coronary Lesions i...,/pubmed/31938934,PURPOSE OF REVIEW: Physiological assessment of...,"Chowdhury M, Osborn EA.",Current treatment options in cardiovascular me...,Current treatment options in cardiovascular me...,PubMed,"Journal Article, Review",PMID:31938934,pubmed,31938934,create date: 2020/01/15 | first author: Chowdh...
3,011620-185722,"(""machine learning"" OR ""deep learning"" OR ""art...",Of Machines and Men: Intelligent Diagnosis and...,/pubmed/31938923,"Artificial Intelligence (AI), although well es...","Cockcroft J, Avolio A.",Current hypertension reports. doi: 10.1007/s11...,Current hypertension reports. 2020,PubMed,"Journal Article, Review",PMID:31938923,pubmed,31938923,create date: 2020/01/14 | first author: Cockcr...
4,011620-185724,"(""machine learning"" OR ""deep learning"" OR ""art...",Cytosine and adenine base editing of the brain...,/pubmed/31937940,The success of base editors for the study and ...,"Levy JM, Yeh WH, Pendse N, Davis JR, Hennessey...",Nature biomedical engineering. doi: 10.1038/s4...,Nature biomedical engineering. 2020,PubMed,Journal Article,PMID:31937940,pubmed,31937940,create date: 2020/01/14 | first author: Levy JM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,011620-194624,"(""machine learning"" OR ""deep learning"" OR ""art...",Lung Innate Lymphoid Cell Composition Is Alter...,/pubmed/31394048,Comment in\n Am J Respir Crit Care Med. 202...,"Monticelli LA, Diamond JM, Saenz SA, Tait Wojn...",American journal of respiratory and critical c...,American journal of respiratory and critical c...,PubMed,Journal Article,PMID:31394048,pubmed,31394048,create date: 2020/01/01 | first author: Montic...
974,011620-195203,"(""machine learning"" OR ""deep learning"" OR ""art...",Curcumin: a therapeutic strategy in cancers by...,/pubmed/31331376,Numerous studies have presented that curcumin ...,"Vallée A, Lecarpentier Y, Vallée JN.",Journal of experimental &amp; clinical cancer ...,Journal of experimental &amp; clinical cancer ...,PubMed,"Journal Article, Review",PMID:31331376,pubmed,31331376,create date: 2019/07/22 | first author: Vallée A
976,011620-195211,"(""machine learning"" OR ""deep learning"" OR ""art...",Extra Virgin Olive Oil Contains a Phenolic Inh...,/pubmed/31331073,The lysine-specific histone demethylase 1A (LS...,"Cuyàs E, Gumuzio J, Lozano-Sánchez J, Carreras...",Nutrients. pii: E1656. doi: 10.3390/nu11071656...,Nutrients. 2019,PubMed,Journal Article,PMID:31331073,pubmed,31331073,create date: 2019/07/19 | first author: Cuyàs E
977,011620-195219,"(""machine learning"" OR ""deep learning"" OR ""art...",Effect of Vasicinone against Paraquat-Induced ...,/pubmed/31331066,Vasicinone is a quinazoline alkaloid isolated ...,"Ju DT, Sivalingam K, Kuo WW, Ho TJ, Chang RL, ...",Nutrients. pii: E1655. doi: 10.3390/nu11071655...,Nutrients. 2019,PubMed,Journal Article,PMID:31331066,pubmed,31331066,create date: 2019/07/19 | first author: Ju DT


In [56]:
from sklearn.feature_extraction.text import CountVectorizer #https://stackabuse.com/python-for-nlp-topic-modeling/
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(subdf['Abstract'].values.astype('U'))
doc_term_matrix

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['age', 'associated', 'cells', 'use', 'study', 'clinical', 'cell', 'patients', 'risk', 'hf']


Top 10 words for topic #1:
['diabetes', 'levels', 'cancer', 'inflammation', 'cholesterol', 'lipoprotein', 'treatment', 'high', 'ldl', 'risk']


Top 10 words for topic #2:
['failure', 'prediction', 'outcomes', 'mortality', 'ml', 'models', '95', 'heart', 'hf', 'patients']


Top 10 words for topic #3:
['care', 'hiv', 'machine', 'results', 'using', 'patients', 'based', 'risk', 'data', 'cardiac']


Top 10 words for topic #4:
['using', '05', 'cells', 'disease', 'inflammatory', 'group', 'expression', 'models', 'patients', 'il']




In [58]:
import gensim #conda install -c conda-forge gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(subdf['Abstract'].values.astype('U'), num_topics = NUM_TOPICS, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

ValueError: not enough values to unpack (expected 2, got 1)

In [61]:
subdf['Abstract'].values.astype('U')

from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
# Create a corpus from a list of texts
common_dictionary = Dictionary(subdf['Abstract'].values)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string